In [ ]:
import anndata as ad
import episcanpy.api as epi
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sns
import time

import episcanpy
print(episcanpy.__version__)

In [ ]:
# download the data from 10x website
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz.tbi
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_singlecell.csv
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_peaks.bed

In [ ]:
DATADIR = ''
peak_file = 'atac_v1_pbmc_5k_peaks.bed' # annotation 
tsv_file = 'atac_v1_pbmc_5k_fragments.tsv.gz' # fragment file
csv_file = 'atac_v1_pbmc_5k_singlecell.csv'  # to filter bad barcodes according to cellranger
output_mtx = 'atac_v1_pbmc_5k_singlecell_peaks.h5ad'

In [ ]:
all_CHR=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
         '15', '16', '17', '18', '19','20', '21', '22', 'X', 'Y', 'M']

WINDOWFILE=peak_file
BEDFILE = tsv_file
OUTPUTPREFIX = 'atac_v1_pbmc_5k_singlecell_peaks_'

chrom_windows = {}
THREAD = 2
#manager = Manager()
#allmtx = manager.dict()


for CHR in all_CHR:
    gen_windows = {}
    gen_windows[CHR] = []
    file = open(WINDOWFILE)
    for line in file:
        ar = line.strip().split('\t')
        if ar[0] == ('chr' + CHR):
            gen_windows[CHR].append([int(ar[1]), int(ar[2])])
    chrom_windows[CHR] = {CHR: gen_windows[CHR]}

In [ ]:
for CHR in all_CHR:
    epi.ct.bld_mtx_fly(bed_file=BEDFILE,
                annotation=chrom_windows[CHR],
                chrom = CHR,
                thread=6,
                save=OUTPUTPREFIX+'_'+CHR+'.h5ad')

In [ ]:
all_CHR=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
         '15', '16', '17', '18', '19','20', '21', '22', 'X', 'Y', 'M']
adata = ad.read('atac_v1_pbmc_5k_singlecell_peaks__'+'1'+'.h5ad')
adata = adata.transpose()
print(adata)


for chrom in all_CHR[1:-1]:
    adata2 = ad.read('atac_v1_pbmc_5k_singlecell_peaks__'+chrom+'.h5ad')
    adata2 = adata2.transpose()
    adata = adata.concatenate(adata2)
    adata.obs_names = [n.split('-')[0] for n in adata.obs_names.tolist()]
    
adata = adata.transpose()
print(adata)

adata.write('atac_v1_pbmc_5k_singlecell_peaks_all_chrom.h5ad')